In [1]:
# find all issues with pdf file
has_file = []
for pi in PrintIssue.objects.all():
    try:
        data = pi.get_page_text_content(2)
    except FileNotFoundError:
        pass
    else:
        has_file.append(pi)

In [4]:
import re
import pprint
positions = 'redaktør', 'desksjef', 'nyhetsleder', 'nettredaktør', 'fotosjef', 'debattredaktør', 'magasinredaktør'
pattern =  re.compile(r'^(?P<title>\w+):\s(?P<name>[-\w ]+)$', flags = re.M | re.I)
def find_positions(issue):
    content = issue.get_page_text_content(2,100)
    #print(content)
    matches = pattern.finditer(content)
    return {d['title']: d['name'] for d in [m.groupdict() for m in matches] if d['title'] in positions}

def add_to_stint(title, name, date):
    try:
        contributor = Contributor.objects.get(display_name=name)
    except Contributor.DoesNotExist:
        print(f'could not find {name}')
        return
    position, _ = Position.objects.get_or_create(title=title)
    defaults = {'start_date': date, 'end_date': date}
    stint, _ = Stint.objects.get_or_create(defaults=defaults, position=position, contributor=contributor)
    stint.end_date = max(stint.end_date, date)
    stint.start_date = min(stint.start_date, date)
    stint.save()
     
    
for issue in has_file:
    data = find_positions(issue)
    content = issue.get_page_text_content(2,100)
    print(issue)
    pprint.pprint(data)
    date = issue.issue.publication_date
    for title, name in data.items():
        add_to_stint(title, name, date)
    


/media/pdf/1715770e623f11e78e030242ac120005/universitas_2016-2.pdf
{'debattredaktør': 'Anders Veberg',
 'desksjef': 'Lise Blekastad',
 'magasinredaktør': 'Ingri Bergo',
 'nettredaktør': 'Signe Rosenlund-Hauglid',
 'nyhetsleder': 'Torgeir Mortensen',
 'redaktør': 'Magnus Newth'}
/media/pdf/1a0660f4623f11e78e030242ac120005/universitas_2016-8.pdf
{'debattredaktør': 'Hanna Skotheim',
 'desksjef': 'Lise Blekastad',
 'magasinredaktør': 'Ingrid Bergo',
 'nettredaktør': 'Signe Rosenlund-Hauglid',
 'redaktør': 'Magnus Newth'}
could not find Ingrid Bergo
/media/pdf/3062444e623f11e78e030242ac120005/universitas_2016-4_mag.pdf
{}
/media/pdf/3270097e623f11e78e030242ac120005/universitas_2016-7.pdf
{'debattredaktør': 'Hanna Skotheim',
 'desksjef': 'Lise Blekastad',
 'magasinredaktør': 'Ingri Bergo',
 'nettredaktør': 'Signe Rosenlund-Hauglid',
 'nyhetsleder': 'Torgeir Mortensen',
 'redaktør': 'Magnus Newth'}
/media/pdf/34f04218623f11e78e030242ac120005/universitas_2016-8_mag.pdf
{}
/media/pdf/1c72356662

/media/pdf/6180eae4623f11e78e030242ac120005/universitas_2016-24.pdf
{'debattredaktør': 'Knut Arne Oseid',
 'desksjef': 'Lise Blekastad',
 'fotosjef': 'Adrian Nielsen',
 'magasinredaktør': 'Kaja Storrøsten',
 'nettredaktør': 'Sondre Myhre',
 'nyhetsleder': 'Birk Tjeldflaat Helle',
 'redaktør': 'Torgeir Mortensen'}
/media/pdf/4c4fa7f0623f11e78e030242ac120005/universitas_2016-16.pdf
{'debattredaktør': 'Hanna Skotheim',
 'desksjef': 'Lise Blekastad',
 'magasinredaktør': 'Ingri Bergo',
 'nettredaktør': 'Signe Rosenlund-Hauglid',
 'nyhetsleder': 'Torgeir Mortensen',
 'redaktør': 'Magnus Newth'}
/media/pdf/40d69406623f11e78e030242ac120005/universitas_2016-18_mag.pdf
{}
/media/pdf/7f63f2ae623f11e78e030242ac120005/universitas_2017-1.pdf
{'debattredaktør': 'Knut Arne Oseid',
 'desksjef': 'Lise Blekastad',
 'fotosjef': 'Adrian Nielsen',
 'magasinredaktør': 'Signe Rosenlund-Hauglid',
 'nettredaktør': 'Sondre Myhre',
 'nyhetsleder': 'Birk Tjeldflaat Helle',
 'redaktør': 'Torgeir Mortensen'}
/media/

In [5]:
Byline.objects.filter(story__story_type__section__title='Kultur', credit='by').update(credit='text')
Byline.objects.filter(story__story_type__section__title='Anmeldelser', credit='by').update(credit='text')
Byline.objects.filter(story__story_type__section__title='Nyhet', credit='by').update(credit='text')
Byline.objects.filter(story__story_type__section__title='Debatt', credit='text').update(credit='by')

163

In [6]:
def add_stints_from_bylines(title, credit, exclude=['Debatt']):
    """Add stints based on byline credits"""
    position, _ = Position.objects.get_or_create(title=title)
    for person in Contributor.objects.all():
        bylines = person.byline_set.filter(
            credit__contains=credit
        ).exclude( # unpublished
            story__publication_date=None
        ).exclude( # no opinion pieces
            story__story_type__section__title__in=exclude
        ).order_by(
            'story__publication_date'
        )
        if bylines.count() > 1:  # ignore people with just one byline
            first_byline = bylines.first().story.publication_date.date()
            last_byline = bylines.last().story.publication_date.date()
            stint, created = Stint.objects.get_or_create(
                dict(start_date=first_byline, end_date=last_byline),
                contributor=person, position=position)
            if not created:
                stint.start_date = min(stint.start_date, first_byline)
                stint.end_date = max(stint.end_date, last_byline)
                stint.save()
            print(f'{stint} {bylines.count()}')
       
    
add_stints_from_bylines('journalist', 'text')
add_stints_from_bylines('fotograf', 'photo')
add_stints_from_bylines('oversetter', 'translation')
add_stints_from_bylines('illustratør', 'illustration')

journalist Hanna Skotheim 10
journalist Benedicte Tobiassen 6
journalist Maria Terese Kittilsen 10
journalist Journalist I Universitas 8
journalist Sunniva R. Skjeggestad 5
journalist Jonas Øren 7
journalist Torgeir Mortensen 51
journalist Tika Sofia León 8
journalist Mads Randen 37
journalist Ida Lyngstad Wernø 8
journalist Louisa Boulaziz 10
journalist Regine Stokstad 24
journalist Benjamin Aarø 3
journalist Zjenja Amundsen 5
journalist Benedicte Tandsæther-Andersen 15
journalist Caroline Ulvin Johansson 3
journalist Kristian Voldnes 5
journalist Ingeborg Misje 14
journalist Brita Søreng 14
journalist Morten Oftedal Schwencke 25
journalist Sondre Myhre 29
journalist Runar B. Mæland 19
journalist Ingeborg Grindheim Slinde 19
journalist Carina I. Hunshamar 10
journalist Patrick da Silva Sæther 3
journalist Ida Wammer 8
journalist Malin Kvande 2
journalist Kitty Byng 2
journalist Hedda Faldet Østberg 2
journalist Håvard Røsæg 16
journalist Kenneth Haug 7
journalist Knut Ward Heimdal 7
j

In [7]:
from django.utils import timezone

def update_active(inactive_limit=timezone.timedelta(days=150)):
    today = timezone.now().date()
    for contributor in Contributor.objects.all():
        all_bylines = contributor.byline_set.exclude(story__publication_date=None)
        debatt_bylines = all_bylines.filter(story__story_type__section__title='Debatt')
        status = Contributor.UNKNOWN
        if all_bylines.count() == 0:
            print(contributor)
        elif all_bylines.count() == debatt_bylines.count():
            status = contributor.EXTERNAL
        elif contributor.stint_set.count():
            latest = contributor.stint_set.latest('end_date')
            if today - latest.end_date > inactive_limit:
                status = contributor.RETIRED
            else:
                status = contributor.ACTIVE
        if status != contributor.status:
            contributor.status=status 
            contributor.save(update_fields=['status'])
        
        print(f'{str(contributor):<40} {contributor.get_status_display():<10} {contributor.byline_set.count():>5}')

update_active()
unknowns = Contributor.objects.filter(status=Contributor.UNKNOWN)
active = Contributor.objects.filter(status=Contributor.ACTIVE)
retired = Contributor.objects.filter(status=Contributor.RETIRED)
print(unknowns.count(), active.count(), retired.count())

Hans Christian Paulsen                   Ekstern        7
Jan Frich                                Ekstern        3
Hanna Skotheim                           Slutta        12
Benedicte Tobiassen                      Aktiv          6
Hans Dalane-Hval                         Ukjent         1
Thorbjørn Kringlebotn Borlaug            Ukjent         1
Haakon Jamtli Kristiansen                Ukjent         1
Ørjan Ryland                             Ukjent         1
Kjartan Hverven                          Ekstern        1
Alexander John Smith-Hald                Ekstern        1
pia sandved berg                         Ukjent         1
Maria Terese Kittilsen                   Slutta        11
Journalist I Universitas                 Slutta        10
Sunniva R. Skjeggestad                   Slutta         5
Jonas Øren                               Slutta         7
Torgeir Mortensen                        Aktiv         62
Svein Otto Nilsen                        Ekstern        1
Carl Henrik Gö

Bente Thorsen                            Ekstern        1
Lill Harriet Sandaune                    Ekstern        1
Margrethe Voll Storaas                   Ekstern        3
Angelique Culvin                         Aktiv         20
Mats Johansen Beldo                      Ekstern        1
Ingvil Høystad Stavem                    Ekstern        1
Ingrid Uleberg                           Ekstern        1
Sylvia-Angélique Culvin                  Slutta         2
Margrete Løbben Hanssen                  Ekstern        1
Mathilde Oda Meyer Jørgensen             Slutta         2
Guro Gravem Johansen                     Ukjent         2
Magnus Ekeland                           Aktiv         23
Hans Ole Rian                            Ukjent         2
Christian Anton Smedshaug                Ekstern        1
Pål Adrian Ryen                          Ekstern        1
Svein Stølen                             Ekstern        2
Markus Slettholm                         Aktiv         42
Hanne Marie Le

Pia Ottilia Danielsen                    Ekstern        1
Tobias Lund                              Ekstern        1
Knut Arne Oseid                          Aktiv         29
Jon Eskil Ramsrud Kindberg               Ekstern        1
Adrian Nielsen                           Aktiv         75
Tobias Grimstad                          Ekstern        1
Hedda Weston                             Ukjent         1
Philip André Johannesborg                Aktiv         73
Alf Grønli Simensen                      Aktiv         11
Henrik Gi
Henrik Gi                                Ukjent         1
Ingrid N. Ekeberg                        Ukjent         2
Hashem Jafari
Hashem Jafari                            Ukjent         1
Motmakt Blindern                         Ekstern        1
Venstrealliansen                         Ekstern        1
SAIH Blindern                            Ekstern        1
Palestinakomiteen Blindern               Ekstern        1
African Student Assosiation              Ekstern

In [25]:
for person in unknowns:
    print(person)
    for stint in person.stint_set.all():
        print(stint)
    for byline in person.byline_set.all():
        print(byline, byline.story.publication_date, byline.story.section)

Emil Øversveen
@bl: Tekst: Emil Øversveen 2015-11-01 18:39:00+00:00 Anmeldelser
Stine Marie Korsfur
@bl: Tekst: Stine Marie Korsfur 2015-11-05 21:19:00+00:00 Nyheter
pia sandved berg
@bl: Tekst: pia sandved berg, Review editor 2016-03-14 10:16:13.412246+00:00 Anmeldelser
Se–Man
@bl: Av: Se–Man 2000-12-26 23:17:00+00:00 Baksiden
@bl: Av: Se–Man 2000-12-26 23:17:00+00:00 Baksiden
Lady Stephanie Huerta Hernandez
@bl: Foto: Lady Stephanie Huerta Hernandez 2015-12-02 14:28:00+00:00 Nyheter
Vilde Boberg Svineng
@bl: Tekst: Vilde Boberg Svineng 2016-02-22 13:06:05.787635+00:00 Anmeldelser
Journalist Malin Kvande
@bl: Tekst: Journalist Malin Kvande 2016-03-28 21:50:00+00:00 Nyheter
Caroline Ulv Johansson
@bl: Tekst: Caroline Ulv Johansson 2016-05-19 12:46:50.200222+00:00 Kultur
Øyunn Syrstad Høydal
@bl: Tekst: Øyunn Syrstad Høydal 2000-12-26 23:17:00+00:00 Kultur
Trygve Bernhardt Moen Haaland
@bl: Av: Trygve Bernhardt Moen Haaland, Formand, Det Norske Studentersamfund 2016-09-12 07:41:00+00:00

@bl: Av: Lars Gule, Studieleder, Høyskolen i Oslo 2007-03-06 23:17:00+00:00 Debatt
@bl: Av: Lars Gule, Postdoktor stipendiat, Høyskolen i Oslo 2011-01-18 23:17:00+00:00 Debatt
Aurora B
@bl: Av: Aurora B 2002-11-19 23:17:00+00:00 Magasin
Too Much Football Woman
@bl: Av: Too Much Football Woman 2002-11-05 23:17:00+00:00 Magasin
F. Nonsens
@bl: Av: F. Nonsens 2003-01-14 23:17:00+00:00 Magasin
Åsmund Øvstehus & Annette Orre
@bl: Tekst: Åsmund Øvstehus & Annette Orre 2003-02-11 23:17:00+00:00 Nyheter
Eivind Yggeseth
@bl: Foto: Eivind Yggeseth 2003-02-18 23:17:00+00:00 Kultur
Anders Aasbø
@bl: Foto: Anders Aasbø 2003-02-18 23:17:00+00:00 Kultur
Dr. Nibbitz
@bl: Av: Dr. Nibbitz 2004-02-17 23:17:00+00:00 Magasin
@bl: Av: Dr. Nibbitz 2004-08-10 23:17:00+00:00 Magasin
@bl: Av: Dr. Nibbitz 2004-11-09 23:17:00+00:00 Magasin
Matrosen
@bl: Av: Matrosen 2004-02-24 23:17:00+00:00 Magasin
Mr. Gjøn
@bl: Av: Mr. Gjøn 2004-03-09 23:17:00+00:00 Magasin
@bl: Av: Mr. Gjøn 2004-11-09 23:17:00+00:00 Magasin
Da

@bl: Av: Aldrende 2006-10-03 23:17:00+00:00 Magasin
Jenny Staurset
@bl: Av: Jenny Staurset 2006-10-03 23:17:00+00:00 Magasin
Audere Est Facere
@bl: Av: Audere Est Facere 2006-10-10 23:17:00+00:00 Magasin
Bukken Von Havresekk
@bl: Av: Bukken Von Havresekk 2006-10-16 23:17:00+00:00 Magasin
Herr Feil
@bl: Av: Herr Feil 2006-10-24 23:17:00+00:00 Magasin
Samselius
@bl: Av: Samselius 2006-10-31 23:17:00+00:00 Magasin
Niklas Lello
@bl: Foto: Niklas Lello 2006-10-31 23:17:00+00:00 Anmeldelser
«Johs. Brun»
@bl: Av: «Johs. Brun» 2006-10-31 23:17:00+00:00 Magasin
Clare Thornton
@bl: Av: Clare Thornton 2006-10-31 23:17:00+00:00 Magasin
Eirin Kongshaug
@bl: Av: Eirin Kongshaug 2006-10-31 23:17:00+00:00 Magasin
@bl: Av: Eirin Kongshaug 2008-02-06 23:17:00+00:00 Magasin
@bl: Av: Eirin Kongshaug, (sminke) 2008-05-27 23:17:00+00:00 Magasin
Nestegangknekkerjegfingradine
@bl: Av: Nestegangknekkerjegfingradine 2006-11-07 23:17:00+00:00 Magasin
Stentrollet Fra Västergötland
@bl: Av: Stentrollet Fra Västerg

@bl: Av: Hanne Prøis Kristiansen 2009-03-31 23:17:00+00:00 Magasin
Gaute Juveth
@bl: Foto: Gaute Juveth 2009-04-21 23:17:00+00:00 Nyheter
Kjell Erik Søvde
@bl: Illustrasjon: Kjell Erik Søvde 2009-04-28 23:17:00+00:00 Feature
Kamilla Simonnes Therese Bongard
@bl: Foto: Kamilla Simonnes Therese Bongard 2009-05-05 23:17:00+00:00 Magasin
For Lån Av Lokaler
@bl: Tekst: For Lån Av Lokaler 2009-05-05 23:17:00+00:00 Magasin
Therese Bongard
@bl: Foto: Therese Bongard 2009-05-05 23:17:00+00:00 Nyheter
Serena Chiara
@bl: Av: Serena Chiara 2009-05-12 23:17:00+00:00 Magasin
Fjodor Sobotka
@bl: Av: Fjodor Sobotka 2009-05-19 23:17:00+00:00 Magasin
Paal Audestad
@bl: Foto: Paal Audestad 2009-08-11 23:17:00+00:00 Feature
Juan Antonio
@bl: Av: Juan Antonio 2009-08-18 23:17:00+00:00 Magasin
Bjørn Ulvang
@bl: Av: Bjørn Ulvang 2009-02-17 23:17:00+00:00 Magasin
Hjernedød
@bl: Av: Hjernedød 2009-10-06 23:17:00+00:00 Magasin
John Lennon
@bl: Av: John Lennon 2009-11-17 23:17:00+00:00 Magasin
Erebos
@bl: Av: Er

@bl: Tekst: Tone Vesterhus, leder av Velferdstinget 2013-10-09 23:17:00+00:00 Nyheter
@bl: Av: Tone Vesterhus, leder av Velferdstinget i Oslo og Akershus 2013-01-15 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, leder av Velferdstinget i Oslo og Akershus 2013-02-26 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, leder av Velferdstinget i Oslo og Akershus 2013-01-15 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, leder av Velferdstinget i Oslo og Akershus 2013-02-26 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, Leder av Velferdstinget i Oslo og Akershus 2013-05-21 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, Leder av Velferdstinget i Oslo og Akershus 2013-05-21 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, Leder av Velferdstinget i Oslo og Akerhus 2013-08-13 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, leder av Velferdstinget i Oslo og Akershus 2013-09-03 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, leder for Velferdstinget i Oslo og Akershus 2013-11-12 23:17:00+00:00 Debatt
@bl: 

In [9]:
# Fjern ubrukte artikkeltyper

for st in StoryType.objects.all():
    if st.story_set.count() == 0:
        print(st)
        st.delete()
        
for sec in Section.objects.all():
    story_types = sec.storytype_set.all()
    if story_types.count() == 0:
        sec.delete()
    print(sec)
    for story_type in story_types:
        print('   ', story_type, story_type.story_set.count())

Nyheter
    Nett 31
    News 36
    Utenriks 45
    Nyhet 447
Magasin
    Magasin 63
Kultur
    Akademia 8
    Kultur 90
Debatt
    Debatt 9
    Leserbrev 195
    Kronikk 29
Baksiden
    Ad Notam 1
    Vi spør 3
Side 2
    Øyeblikket 1
    Kommentar 51
    Leder 4
Anmeldelser
    Anmeldelse 243
Feature
    Reportasje 5
    Studietid 10
    Min studietid 19
Annet
    Universitas Melder 2
    Quiz 5
    New Story Type (kulturnotis) 2
